In [1]:
## CS 207 - Group 9
## Demo for Project checkpoint 1
## Reaction rate calculator

from chemkin import *

reader = xml2dict()
file_path = input('Welcome to the Reaction Rate calculator. \n\nInput a valid xml file: \t')
reader.parse(file_path)
info = reader.get_info()

reactions = [Reaction(**r) for r in info[1]]
species=info[0]
rs = ReactionSystem(reactions,species_ls=species)

str_of_concs = input('input concentrations, separated by commas, in the following order:\n '+str(species)+'\t')
array_of_concs = [float(i) for i in str_of_concs.split(',')]
concs = dict(zip(species,array_of_concs))
print(concs)
rs.set_concs(concs)

temp_string = input('input non-negative temperature in Kelvins: \t')
rs.set_temp(float(temp_string))

rs.get_reac_rate()
print('\nReaction rate for species', species,
      '\nwith concentrations      ', array_of_concs, 
      '\nat temperature           ', temp_string, '\nis, in order:')
print(rs.get_reac_rate())

print('\nProgress rate for reactions with concentrations', array_of_concs, 
      'at temperature', temp_string, 'is:')
print(rs.get_progress_rate())


Welcome to the Reaction Rate calculator. 

Input a valid xml file: 	rxns2.xml
input concentrations, separated by commas, in the following order:
 ['H2', 'O2', 'OH', 'HO2', 'H2O']	2.0, 1.0, 0.5, 1.0, 1.0
{'H2': 2.0, 'O2': 1.0, 'OH': 0.5, 'HO2': 1.0, 'H2O': 1.0}
input non-negative temperature in Kelvins: 	750

Reaction rate for species ['H2', 'O2', 'OH', 'HO2', 'H2O'] 
with concentrations       [2.0, 1.0, 0.5, 1.0, 1.0] 
at temperature            750 
is, in order:
[-3607077.87280406 -5613545.18362079  9220623.05642485  2006467.31081673
 -2006467.31081673]

Progress rate for reactions with concentrations [2.0, 1.0, 0.5, 1.0, 1.0] at temperature 750 is:
[ 3607077.87280406     5000.          2011467.31081673]


## Create a Reaction and visualize it

In [3]:
from chemkin import *

reaction_demo = Reaction(
    reactants=dict(H2=1,O2=1), 
    products=dict(OH=2),
    coeffLaw='Arrhenius', 
    coeffParams=dict(E=-4.157)
)
print(reaction_demo)

Reaction Equation:
1H2 + 1O2 => 2OH
----------------------------------------
Reaction Info:
ID: reaction
TYPE: Elementary
reversible: False
coeffLaw: Arrhenius
coeffParams: [('A', 1.0), ('E', -4.157), ('R', 8.314)]
coeffUnits: []


## Batch import from a XML file to build a ReactionSystem

In [4]:
xml_reader = xml2dict()
xml_reader.parse('rxns2.xml')
species, reaction_info_list = xml_reader.get_info()

reaction_list = [Reaction(**rinfo) for rinfo in reaction_info_list]
reaction_system = ReactionSystem(reaction_list, species_ls=species)

print('Species involved: \n', species,'\n\n')
print(reaction_system)

Species involved: 
 ['H2', 'O2', 'OH', 'HO2', 'H2O'] 


ReactionSystem object with following Reactions: 

Reaction 0: 
Reaction Equation:
2H2 + 1O2 => 1H2 + 2OH
----------------------------------------
Reaction Info:
ID: reaction01
TYPE: Elementary
reversible: False
coeffLaw: modArrhenius
coeffParams: [('A', 100000000.0), ('E', 50000.0), ('R', 8.314), ('b', 0.5)]
coeffUnits: []

Reaction 1: 
Reaction Equation:
1HO2 + 1OH => 1H2O + 1O2
----------------------------------------
Reaction Info:
ID: reaction02
TYPE: Elementary
reversible: False
coeffLaw: Constant
coeffParams: [('k', 10000.0)]
coeffUnits: []

Reaction 2: 
Reaction Equation:
1H2O + 1O2 => 1HO2 + 1OH
----------------------------------------
Reaction Info:
ID: reaction03
TYPE: Elementary
reversible: False
coeffLaw: Arrhenius
coeffParams: [('A', 10000000.0), ('E', 10000.0), ('R', 8.314)]
coeffUnits: []



## Set temp/concs at ANY time

In [5]:
# species order: 
# ['H2', 'O2', 'OH', 'HO2', 'H2O'] 

concs = [2.0, 1.0, 0.5, 1.0, 1.0]
Temps = [750.0, 1500.0, 2500.0]

reaction_system.set_concs(dict(zip(species, concs)))
reaction_system.set_temp(Temps[0])

print('Concentration by species: \n', reaction_system.get_concs())
print('\nTemperature:', reaction_system.get_temp(), 'Kelvin')

Concentration by species: 
 {'H2': 2.0, 'O2': 1.0, 'OH': 0.5, 'HO2': 1.0, 'H2O': 1.0}

Temperature: 750.0 Kelvin


## Caculate the reaction rate:  Accuracy, Efficiency, Elegancy

In [6]:
reaction_rates = reaction_system.get_reac_rate()
print('Reaction rate for species', species, 'is, in order:')
print(['{:8.4e}'.format(rate) for rate in reaction_rates])

progress_rates = reaction_system.get_progress_rate()
print('\nProgress rate for involved reactions is, in order:')
print(['{:8.4e}'.format(rate) for rate in progress_rates])

Reaction rate for species ['H2', 'O2', 'OH', 'HO2', 'H2O'] is, in order:
['-3.6071e+06', '-5.6135e+06', '9.2206e+06', '2.0065e+06', '-2.0065e+06']

Progress rate for involved reactions is, in order:
['3.6071e+06', '5.0000e+03', '2.0115e+06']


## Infrastructure: the LAW for reaction rate coeffs

In [7]:
eval(repr(reaction_demo))

{'ID': 'reaction',
 'TYPE': 'Elementary',
 'coeffLaw': 'Arrhenius',
 'coeffParams': {'A': 1.0, 'E': -4.157, 'R': 8.314},
 'coeffUnits': {},
 'products': {'OH': 2},
 'reactants': {'H2': 1, 'O2': 1},
 'reversible': False}

In [8]:
print(reaction_demo.rateCoeff(check=True, T=0.5))

2.718281828459045


In [9]:
reaction_demo.set_params(
    coeffLaw='NewLaw', 
    coeffParams=dict(E=-4.157e10, R=8.314)
)
reaction_demo.rateCoeff(check=True, T=-2.0)

NotImplementedError: coeffLaw = NewLaw. Referred reaction rate coefficient law is not implemented.

In [10]:
Reaction._CoeffLawDict.getcopy_all()

{'Arrhenius': chemkin.Arrhenius,
 'Constant': chemkin.Constant,
 'modArrhenius': chemkin.modArrhenius}

## Extensibility?

In [12]:
class NewLaw(MathModel):
    
    @staticmethod
    def _kernel(T, E, R):
        return -E / (T * R)
    
#     @staticmethod
#     def check_coeffparams(R, **other_params):
#         if R <= 0.0:
#             raise ValueError(
#                 'R = {:18.16e}: non-positive R is invalid.'.format(R))
#     def check_stateparams(self, T, **other_params):
#         if T <= 0.0:
#             raise ValueError(
#                 'T = {:18.16e}: non-positive T is invalid.'.format(T))

Reaction._CoeffLawDict.update('NewLaw', NewLaw)
Reaction._CoeffLawDict.getcopy_all()

{'Arrhenius': chemkin.Arrhenius,
 'Constant': chemkin.Constant,
 'NewLaw': __main__.NewLaw,
 'modArrhenius': chemkin.modArrhenius}

In [13]:
reaction_demo.set_params(
    coeffLaw='NewLaw', 
    coeffParams=dict(E=-4.157e10, R=8.314)
)
reaction_demo.rateCoeff(check=True, T=2.0)

2500000000.0

In [14]:
new_reaction_system = ReactionSystem(reaction_list+[reaction_demo], species)
new_reaction_system.set_concs(dict(zip(species, concs)))
new_reaction_system.set_temp(Temps[0])

new_reaction_rates = new_reaction_system.get_reac_rate()
print('Reaction rate for species', species, 'is, in order:')
print(['{:8.4e}'.format(rate) for rate in new_reaction_rates])

Reaction rate for species ['H2', 'O2', 'OH', 'HO2', 'H2O'] is, in order:
['-1.6940e+07', '-1.8947e+07', '3.5887e+07', '2.0065e+06', '-2.0065e+06']
